# Visualize weather data from https://www.earthsystemgrid.org/dataset/isabeldata.html

Written by Jean M Favre, Swiss National Supercomputing Center.
Tested on Piz Daint Thu 01 Oct 2020 08:38:49 AM CEST

In [ ]:
import gzip
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

# Details about file format are here:

https://www.earthsystemgrid.org/dataset/isabeldata/file/readme.html

http://vis.computer.org/vis2004contest/data.html

Data Format

A summary of the data can be found below:

    Dimensions: 500 x 500 x 100
    Physical Scale: 2139km (east-west) x 2004km (north-south) x 19.8km (vertical) Note: The east-west distance of 2139km is measured at the southern latitude; the distance is only 1741km at the northern latitude due to the curvature of the earth.
    Physical Location: Longitude (x): 83W to 62W; Latitude (y): 23.7N to 41.7N; Height (z): 0.035km to 19.835km
    Format: Brick-of-Floats

In [ ]:
dims = [500,500,100] # original dimensions of binary data in disk file

def load_slice(frame_number=1, K=50):
  assert frame_number >= 1 and frame_number <= 48
  fname_gzipped = format('/scratch/snx3000/jfavre/Isabel/QVAPORf%02d.bin.gz' % frame_number)
  #print("opening ", fname_gzipped)
  with gzip.open(fname_gzipped,'rb') as f:
    # read a single slice in the Z direction by slicing the volume
    data = np.frombuffer(f.read(), dtype='>f4')[dims[0]*dims[1]*K:dims[0]*dims[1]*(K+1)]
    # Land values, where there is no valid atmospheric data, are marked 1.0e35
    data = np.where(data!=1e35, data, np.NaN)
    data = data.reshape(dims[0:2])
    data = np.fliplr(np.rot90(data,-1))
  return data

In [ ]:
def animate():
    def display_slice_in_time(time=48):
        plt.imshow(load_slice(time), extent=[83, 62,23.7, 41.7], cmap=plt.get_cmap("PuBuGn"))
        plt.title('Timestep: %d' % time)
        plt.show()
    interact(display_slice_in_time, time=(1,48))
    
animate()